In [1]:
import numpy as np
import open3d as o3d
import os, shutil
import yaml
import gc
import copy
from tqdm import tqdm_notebook
from multiprocessing import Pool, cpu_count

In [2]:
# CFG_PATH = "config/semantic-kitti.yaml"
VELODYNE_PATH = "/home/saby/Projects/ati/data/data/datasets/KITTI/dataset"
LABELS_PATH = "/home/saby/Projects/ati/data/data/datasets/KITTI/data_odometry_labels/dataset"
# SEQUENCE_NO_FOLDER = "09"

SEQUENCE_FOLDER = "sequences"
LIDAR_FOLDER = "velodyne"
LABEL_FOLDER = "labels"
SEGMENT_OUT_FOLDER = "_segment_out_npy"

LABELS_SEQUENCE_PATH = os.path.join(LABELS_PATH, SEQUENCE_FOLDER)

In [3]:
def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def make_npy(raw_lidar, raw_label):
#     some_arr = raw_lidar[:,:3]
    transformed_raw_lidar = np.zeros(raw_lidar.shape)
    transformed_raw_lidar[:,0] = copy.deepcopy(raw_lidar[:,1])
    transformed_raw_lidar[:,1] = -copy.deepcopy(raw_lidar[:,0])
    transformed_raw_lidar[:,2] = -copy.deepcopy(raw_lidar[:,2])
#     some_arr = transformed_raw_lidar[:,:3]
    
    raw_label = np.array([label & 0xFFFF for label in raw_label])
    raw_label = np.array([label2mask_dict[label] for label in raw_label])
    transformed_raw_lidar[:,3] = raw_label
#     color_arr = np.concatenate((raw_label, np.zeros(raw_label.shape), np.zeros(raw_label.shape)), axis=1)
    
#     some_pcd = o3d.geometry.PointCloud()
#     some_pcd.points = o3d.utility.Vector3dVector(some_arr)
#     some_pcd.colors = o3d.utility.Vector3dVector(color_arr)
    
    return transformed_raw_lidar

In [4]:
label2mask_dict = {0: 0,#'unlabeled',
                     1: 0,#'outlier',
                     10: 1,#'car',
                     11: 1,#'bicycle',
                     13: 1,#'bus',
                     15: 1,#'motorcycle',
                     16: 1,#'on-rails',
                     18: 1,#'truck',
                     20: 1,#'other-vehicle',
                     30: 1,#'person',
                     31: 1,#'bicyclist',
                     32: 1,#'motorcyclist',
                     40: 0,#'road',
                     44: 0,#'parking',
                     48: 0,#'sidewalk',
                     49: 0,#'other-ground',
                     50: 0,#'building',
                     51: 0,#'fence',
                     52: 0,#'other-structure',
                     60: 0,#'lane-marking',
                     70: 0,#'vegetation',
                     71: 0,#'trunk',
                     72: 0,#'terrain',
                     80: 0,#'pole',
                     81: 0,#'traffic-sign',
                     99: 0,#'other-object',
                     252: 1,#'moving-car',
                     253: 1,#'moving-bicyclist',
                     254: 1,#'moving-person',
                     255: 1,#'moving-motorcyclist',
                     256: 1,#'moving-on-rails',
                     257: 1,#'moving-bus',
                     258: 1,#'moving-truck',
                     259: 1}#'moving-other-vehicle'

In [5]:
def parallel_thread(arg):
    gc.collect()
    lidar_file, label_file = arg[0], arg[1]
    assert int(lidar_file.split(".")[0]) == int(label_file.split(".")[0])
    
    lidar_file_path = os.path.join(LIDAR_PATH, lidar_file)
    label_file_path = os.path.join(LABEL_PATH, label_file)
    
    raw_lidar = np.fromfile(lidar_file_path, dtype=np.float32).reshape((-1, 4))    
    raw_label = np.fromfile(label_file_path, dtype=np.uint32)
    
    this_arr = make_npy(raw_lidar, raw_label)
#     fname = str(int(lidar_file.split(".")[0])) + ".ply"
#     pcd_path = os.path.join(SEGMENT_PATH, fname)
#     o3d.io.write_point_cloud(pcd_path, this_pcd)
    return this_arr

In [6]:
for SEQUENCE_NO_FOLDER in sorted(os.listdir(LABELS_SEQUENCE_PATH)):
    if int(SEQUENCE_NO_FOLDER) != 1:
        continue
    print("Sequence No. ", SEQUENCE_NO_FOLDER)
    LIDAR_PATH = os.path.join(VELODYNE_PATH, SEQUENCE_FOLDER, SEQUENCE_NO_FOLDER, LIDAR_FOLDER)
    LABEL_PATH = os.path.join(LABELS_PATH, SEQUENCE_FOLDER, SEQUENCE_NO_FOLDER, LABEL_FOLDER)
    SEGMENT_OUT_PATH = os.path.join(LABELS_PATH, SEQUENCE_FOLDER, SEQUENCE_NO_FOLDER, SEGMENT_OUT_FOLDER)

    if not os.path.exists(SEGMENT_OUT_PATH):
        os.makedirs(SEGMENT_OUT_PATH)
    else:
        shutil.rmtree(SEGMENT_OUT_PATH)
        os.makedirs(SEGMENT_OUT_PATH)

    LIDAR_FILES = sorted(os.listdir(LIDAR_PATH))
    LABEL_FILES = sorted(os.listdir(LABEL_PATH))

    parallel_args = list(zip(LIDAR_FILES, LABEL_FILES))
    process_pool = Pool(cpu_count()-1)
    segment_list = [each for each in tqdm_notebook(process_pool.imap(parallel_thread, parallel_args),
                                         total = len(parallel_args))]
    process_pool.terminate()
    gc.collect()
    
    segment_arr = np.array(segment_list)
    fname = "0.npy"
    file_path = os.path.join(SEGMENT_OUT_PATH, fname)
    np.save(file_path, segment_arr, allow_pickle)
    gc.collect()

Sequence No.  01


/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
